In [1]:
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import scipy.optimize

In [2]:
import astro_time
import sgp4
import sensor

import tle_fitter

In [3]:
import public_astrostandards as PA
PA.init_all()
# PA.get_versions()

0

In [4]:
# grab some raw UDL obs; convert the time; sort
def prepObs( o_df ):
    o_df['obTime_dt'] = pd.to_datetime( o_df['obTime'] )
    o_df = o_df.sort_values(by='obTime_dt')
    t_df = astro_time.convert_times( obs['obTime_dt'], PA )
    o_df = pd.concat( (t_df.reset_index(drop=True),o_df.reset_index(drop=True) ), axis=1 )
    return o_df 
    
obs    = pd.read_json('./19548.json.gz').sort_values(by='obTime').reset_index(drop=True)
obs_df = prepObs( obs )    

In [5]:
def rotateTEMEObs( O , harness ):
    '''
    given an ob (O) with ra / declination fields (J2K), convert to TEME
    '''
    newRA  = (harness.ctypes.c_double)()
    newDec = (harness.ctypes.c_double)()
    PA.AstroFuncDll.RotRADec_EqnxToDate( 106,
                                        2,
                                        O['ds50_utc'],
                                        O['ra'],
                                        O['declination'],
                                        newRA,
                                        newDec )
    return ( np.float64(newRA),np.float64(newDec) )

# rotate a dataframe of obs into TEME and then also get a TEME look vector (for solving)
def rotateTEMEdf( df, harness ):
    tv = df.apply( lambda X : rotateTEMEObs( X, harness ) , axis=1 )
    df['teme_ra']  = [ X[0] for X in tv ]
    df['teme_dec'] = [ X[1] for X in tv ]
    x = np.cos( np.radians(df['teme_dec']) ) * np.cos( np.radians( df['teme_ra'] ) )
    y = np.cos( np.radians(df['teme_dec']) ) * np.sin( np.radians( df['teme_ra'] ) )
    z = np.sin( np.radians(df['teme_dec'] ) )
    lv = np.hstack( ( x.values[:,np.newaxis], y.values[:,np.newaxis], z.values[:,np.newaxis] )  )
    df['teme_lv'] = lv.tolist()
    return df

obs_df = rotateTEMEdf( obs_df, PA )

In [6]:
L1 = '1 19548U 88091B   25281.05493527 -.00000297  00000+0  00000+0 0  9993'
L2 = '2 19548  12.7961 342.6596 0038175 340.8908  24.1736  1.00278194122860'

In [7]:
# -----------------------------------------------------------------------------------------------------
def optFunction( X, EH, return_scalar=True ):
    PA      = EH.PA
    XS_TLE  = PA.Cstr('',512)
    # take the function parameters (X) and overwrite the "new_tle" values based on FIELDS 
    for k,v in zip(EH.FIELDS,X) :  EH.new_tle[ k ] = v
    # --------------------- clear state
    PA.TleDll.TleRemoveAllSats()
    PA.Sgp4PropDll.Sgp4RemoveAllSats()
    # --------------------- init our test TLE from the modified data
    tleid = PA.TleDll.TleAddSatFrArray( EH.new_tle.data, XS_TLE )
    if tleid <= 0: return np.inf
    if PA.Sgp4PropDll.Sgp4InitSat( tleid ) != 0: return np.inf
    # --------------------- generate our test ephemeris
    test_frame = sgp4.propTLE_byID_df( tleid, EH.date_f, EH.PA )
    looks  =  sensor.compute_looks( EH.sensor_df, test_frame, EH.PA )
    RA_residuals  = EH.obs_df['teme_ra'] - looks['XA_TOPO_RA']
    DEC_residuals =  EH.obs_df['teme_dec'] - looks['XA_TOPO_DEC']
    rv =  np.sum( np.abs(RA_residuals) ) + np.sum( np.abs( DEC_residuals ) )
    print('{:8.3f}                '.format(rv), end='\r')
    return rv

In [24]:
# -----------------------------------------------------------------------------------------------------
class eo_fitter( tle_fitter.tle_fitter ):
    def __init__( self, PA ):
        super().__init__( PA )
        self.line1 = None
        self.line2 = None

    def _prop_to_new_epoch( self, epoch ):
        ''' assume that epoch is set, and that line1, line2 are also set '''
        self.PA.TleDll.TleRemoveAllSats()
        tleid = sgp4.addTLE( self.line1, self.line2, self.PA )
        assert sgp4.initTLE( tleid, self.PA )
        rv    = sgp4.propTLEToDS50s( tleid, [ epoch ], self.PA )[0]
        rv    = { 'teme_p' : rv[1:4], 'teme_v' : rv[4:7], 'ds50_utc' : self.epoch_ds50 }
        return rv

    def set_data( self, L1 : str, L2 : str, obs : list[ dict ] ):
        ''' 
        take an initial TLE as a guess (L1,L2) 
        take a list of JSON formatted obs (directly from UDL)
        solve for a new TLE
        '''
        self.line1      = L1
        self.line2      = L2
        self.obs        = obs

        self.obs_df     = prepObs( obs )
        self.obs_df     = rotateTEMEdf( self.obs_df, self.PA )
        self._look_vecs = np.vstack( self.obs_df['teme_lv'] )
        self.date_f     = self.obs_df[ ['ds50_utc','ds50_et','theta']].copy()

        # init the TLE from the lines data
        self.init_tle    = tle_fitter.TLE_str_to_XA_TLE( L1, L2, self.PA )
        self.new_tle     = tle_fitter.TLE_str_to_XA_TLE( L1, L2, self.PA )
        # self.epoch_ds50  = self.obs_df.iloc[ -1 ]['ds50_utc']
        # epoch_sv         = self._prop_to_new_epoch( self.epoch_ds50 )
        # self.set_from_sv( epoch_sv )
        # self.new_tle['XA_TLE_EPOCH'] = epoch_sv['ds50_utc']
        # # if this is a type-0, we need Kozai mean motion   
        # if self.new_tle['XA_TLE_EPHTYPE'] == 0:
        #     self.new_tle['XA_TLE_MNMOTN'] = self.PA.AstroFuncDll.BrouwerToKozai( 
        #                                         self.new_tle['XA_TLE_ECCEN'], 
        #                                         self.new_tle['XA_TLE_INCLI'],
        #                                         self.new_tle['XA_TLE_MNMOTN'] )

        self.sensor_df        = self.obs_df[['ds50_utc','senlat','senlon','senalt','theta']]
        self.sensor_df        = self.sensor_df.rename( columns = {'senlat' : 'lat','senlon' : 'lon', 'senalt' : 'height' } )
        self.sensor_df        = sensor.llh_to_eci( self.sensor_df, self.PA )
        return self
                                       

obs    = pd.read_json('./19548.json.gz').sort_values(by='obTime').reset_index(drop=True)    
# A = eo_fitter( PA ).set_data( L1, L2, obs ).clear_nonconservatives().set_type4()
A = eo_fitter( PA ).set_type4().set_data( L1, L2, obs )

optFunction( A.get_init_fields(), A )


# -----------------------------  nelder mead -----------------------------
# if your seed is not near the final, nelder works great (at the expense of time)
ans   = scipy.optimize.minimize(optFunction, 
                                A.get_init_fields(),
                                args    = (A,True),
                                method  = 'Nelder-Mead' ,
                                #options = {'xatol' : 0.01, 'fatol' : 0.9 } )
                                options = {'xatol' : 0.01, 'fatol' : 0.1, 'initial_simplex' : A.initial_simplex() } )
                               #options = {'initial_simplex' : smplx } )
if ans.success:
    A.reset_tle()
    # now update with perturbed values (not sure if this is necessary... last step should be there)
    for k,v in zip(A.FIELDS,ans.x) : A.new_tle[k] = v
    A.ans = ans

   3.624                  

In [19]:
nL1,nL2 = A.getLines()
test_eph = sgp4.propTLE_df( A.date_f, nL1, nL2, A.PA )

test_looks = sensor.compute_looks( A.sensor_df, test_eph, A.PA )

In [21]:
3600 * (test_looks['XA_TOPO_RA'] - A.obs_df['teme_ra'])

0        8.644022
1        8.114882
2       11.347952
3        8.670508
4        9.401458
          ...    
1479    20.398925
1480    19.578079
1481    20.075474
1482    19.530194
1483    19.706718
Length: 1484, dtype: float64

In [23]:
3600 * (test_looks['XA_TOPO_DEC'] - A.obs_df['teme_dec'])

0       2.251920
1       1.522662
2       4.106453
3       2.075810
4       1.619867
          ...   
1479   -3.905444
1480   -3.811167
1481   -4.409977
1482   -3.179919
1483   -4.550127
Length: 1484, dtype: float64